# Data prep Events

In [4]:
import pandas as pd

# Lade den existierenden Datensatz
df = pd.read_csv('../_data_raw/df_kiel_events.csv')

# Konvertiere die Spalte 'date' in ein korrektes Datetime-Format
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')

# Entferne die Spalten 'maritime' und 'nordic_film_days', falls sie existieren
columns_to_drop = ['maritime_conference', 'nordic_film_days']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

# Finde das letzte Datum im aktuellen Datensatz
end_date_current = df['date'].max()

# Generiere Datumsbereich bis 30.07.2019
new_dates = pd.date_range(start=end_date_current + pd.Timedelta(days=1), end='2019-07-30')

# Feiertage speziell für Schleswig-Holstein 2018
sh_holidays_2018 = [
    '2018-01-01',   # Neujahr
    '2018-03-30',   # Karfreitag
    '2018-04-01',   # Ostersonntag
    '2018-04-02',   # Ostermontag
    '2018-05-01',   # Tag der Arbeit
    '2018-05-10',   # Christi Himmelfahrt
    '2018-05-21',   # Pfingstmontag
    '2018-06-06',   # Fronleichnam
    '2018-10-03',   # Tag der Deutschen Einheit
    '2018-11-01',   # Reformationstag (in Schleswig-Holstein ein Feiertag)
    '2018-12-25',   # 1. Weihnachtsfeiertag
    '2018-12-26'    # 2. Weihnachtsfeiertag
]

sh_holidays_2019 = [
    '2019-01-01',   # Neujahr
    '2019-04-19',   # Karfreitag
    '2019-04-21',   # Ostersonntag
    '2019-04-22',   # Ostermontag
    '2019-05-01',   # Tag der Arbeit
    '2019-05-30',   # Christi Himmelfahrt
    '2019-06-10',   # Pfingstmontag
]

# Combine both lists
sh_holidays_2018.extend(sh_holidays_2019)

# Neues DataFrame mit zusätzlichen Daten erstellen
new_data = pd.DataFrame({
    'date': new_dates,
    'national_holiday': [1 if str(d.date()) in sh_holidays_2018 else 0 for d in new_dates],
    'christmas_market': [0] * len(new_dates)  # Setze 'christmas_market' auf 0
})

# Zusammenfügen der neuen Daten mit dem existierenden Datensatz
df_extended = pd.concat([df, new_data], ignore_index=True)

# rename column date into Datum in new_data
df_extended.rename(columns={'date': 'Datum'}, inplace=True)

df_extended['Datum'] = pd.to_datetime(df_extended['Datum'], format='%d.%m.%Y').dt.strftime('%Y-%m-%d')


# Speichere den erweiterten Datensatz
df_extended.to_csv('../_Variables_cleaned/df_kiel_events.csv', index=False, date_format='%d.%m.%Y')

